# Searching Unstructured and Structured Data #
## Assignment 1: Retrieval models [100 points] ##
**TA**: Nikos Voskarides (n.voskarides@uva.nl)

In this assignment you will get familiar with basic information retrieval concepts. You will implement and evaluate different information retrieval ranking models and evaluate their performance.

We provide you with a Indri index. To query the index, you'll use a Python package ([pyndri](https://github.com/cvangysel/pyndri)) that allows easy access to the underlying document statistics.

For evaluation you'll use the [TREC Eval](https://github.com/usnistgov/trec_eval) utility, provided by the National Institute of Standards and Technology of the United States. TREC Eval is the de facto standard way to compute Information Retrieval measures and is frequently referenced in scientific papers.

This is a **groups-of-three assignment**, the deadline is **Monday, 22/1, at 23:59**. Code quality, informative comments and convincing analysis of the results will be considered when grading. Submission should be done through blackboard, questions can be asked on the course [Piazza](https://piazza.com/class/ixoz63p156g1ts).

### Technicalities (must-read!) ###

The assignment directory is organized as follows:
   * `./assignment.ipynb` (this file): the description of the assignment.
   * `./index/`: the index we prepared for you.
   * `./ap_88_90/`: directory with ground-truth and evaluation sets:
      * `qrel_test`: test query relevance collection (**test set**).
      * `qrel_validation`: validation query relevance collection (**validation set**).
      * `topics_title`: semicolon-separated file with query identifiers and terms.

You will need the following software packages (tested with Python 3.5 inside [Anaconda](https://conda.io/docs/user-guide/install/index.html)):
   * Python 3.5 and Jupyter
   * Indri + Pyndri (Follow the installation instructions [here](https://github.com/nickvosk/pyndri/blob/master/README.md))
   * gensim [link](https://radimrehurek.com/gensim/install.html)
   * TREC Eval [link](https://github.com/usnistgov/trec_eval)

### TREC Eval primer ###
The TREC Eval utility can be downloaded and compiled as follows:

    git clone https://github.com/usnistgov/trec_eval.git
    cd trec_eval
    make

TREC Eval computes evaluation scores given two files: ground-truth information regarding relevant documents, named *query relevance* or *qrel*, and a ranking of documents for a set of queries, referred to as a *run*. The *qrel* will be supplied by us and should not be changed. For every retrieval model (or combinations thereof) you will generate a run of the top-1000 documents for every query. The format of the *run* file is as follows:

    $query_identifier Q0 $document_identifier $rank_of_document_for_query $query_document_similarity $run_identifier
    
where
   * `$query_identifier` is the unique identifier corresponding to a query (usually this follows a sequential numbering).
   * `Q0` is a legacy field that you can ignore.
   * `$document_identifier` corresponds to the unique identifier of a document (e.g., APXXXXXXX where AP denotes the collection and the Xs correspond to a unique numerical identifier).
   * `$rank_of_document_for_query` denotes the rank of the document for the particular query. This field is ignored by TREC Eval and is only maintained for legacy support. The ranks are computed by TREC Eval itself using the `$query_document_similarity` field (see next). However, it remains good practice to correctly compute this field.
   * `$query_document_similarity` is a score indicating the similarity between query and document where a higher score denotes greater similarity.
   * `$run_identifier` is an identifier of the run. This field is for your own convenience and has no purpose beyond bookkeeping.
   
For example, say we have two queries: `Q1` and `Q2` and we rank three documents (`DOC1`, `DOC2`, `DOC3`). For query `Q1`, we find the following similarity scores `score(Q1, DOC1) = 1.0`, `score(Q1, DOC2) = 0.5`, `score(Q1, DOC3) = 0.75`; and for `Q2`: `score(Q2, DOC1) = -0.1`, `score(Q2, DOC2) = 1.25`, `score(Q1, DOC3) = 0.0`. We can generate run using the following snippet:

In [1]:
import logging
import sys
import os

def write_run(model_name, data, out_f,
              max_objects_per_query=sys.maxsize,
              skip_sorting=False):
    """
    Write a run to an output file.
    Parameters:
        - model_name: identifier of run.
        - data: dictionary mapping topic_id to object_assesments;
            object_assesments is an iterable (list or tuple) of
            (relevance, object_id) pairs.
            The object_assesments iterable is sorted by decreasing order.
        - out_f: output file stream.
        - max_objects_per_query: cut-off for number of objects per query.
    """
    for subject_id, object_assesments in data.items():
        if not object_assesments:
            logging.warning('Received empty ranking for %s; ignoring.',
                            subject_id)

            continue

        # Probe types, to make sure everything goes alright.
        # assert isinstance(object_assesments[0][0], float) or \
        #     isinstance(object_assesments[0][0], np.float32)
        assert isinstance(object_assesments[0][1], str) or \
            isinstance(object_assesments[0][1], bytes)

        if not skip_sorting:
            object_assesments = sorted(object_assesments, reverse=True)

        if max_objects_per_query < sys.maxsize:
            object_assesments = object_assesments[:max_objects_per_query]

        if isinstance(subject_id, bytes):
            subject_id = subject_id.decode('utf8')

        for rank, (relevance, object_id) in enumerate(object_assesments):
            if isinstance(object_id, bytes):
                object_id = object_id.decode('utf8')

            out_f.write(
                '{subject} Q0 {object} {rank} {relevance} '
                '{model_name}\n'.format(
                    subject=subject_id,
                    object=object_id,
                    rank=rank + 1,
                    relevance=relevance,
                    model_name=model_name))
            
# The following writes the run to standard output.
# In your code, you should write the runs to local
# storage in order to pass them to trec_eval.
write_run(
    model_name='example',
    data={
        'Q1': ((1.0, 'DOC1'), (0.5, 'DOC2'), (0.75, 'DOC3')),
        'Q2': ((-0.1, 'DOC1'), (1.25, 'DOC2'), (0.0, 'DOC3')),
    },
    out_f=sys.stdout,
    max_objects_per_query=1000)



Q1 Q0 DOC1 1 1.0 example
Q1 Q0 DOC3 2 0.75 example
Q1 Q0 DOC2 3 0.5 example
Q2 Q0 DOC2 1 1.25 example
Q2 Q0 DOC3 2 0.0 example
Q2 Q0 DOC1 3 -0.1 example


Now, imagine that we know that `DOC1` is relevant and `DOC3` is non-relevant for `Q1`. In addition, for `Q2` we only know of the relevance of `DOC3`. The query relevance file looks like:

    Q1 0 DOC1 1
    Q1 0 DOC3 0
    Q2 0 DOC3 1
    
We store the run and qrel in files `example.run` and `example.qrel` respectively on disk. We can now use TREC Eval to compute evaluation measures. In this example, we're only interested in Mean Average Precision and we'll only show this below for brevity. However, TREC Eval outputs much more information such as NDCG, recall, precision, etc.

    $ trec_eval -m all_trec -q example.qrel example.run | grep -E "^map\s"
    > map                   	Q1	1.0000
    > map                   	Q2	0.5000
    > map                   	all	0.7500
    
Now that we've discussed the output format of rankings and how you can compute evaluation measures from these rankings, we'll now proceed with an overview of the indexing framework you'll use.

### Pyndri primer ###
For this assignment you will use [Pyndri](https://github.com/cvangysel/pyndri) [[1](https://arxiv.org/abs/1701.00749)], a python interface for [Indri](https://www.lemurproject.org/indri.php). We have indexed the document collection and you can query the index using Pyndri. We will start by giving you some examples of what Pyndri can do:

First we read the document collection index with Pyndri:

In [2]:
import pyndri

index = pyndri.Index('index/')

The loaded index can be used to access a collection of documents in an easy manner. We'll give you some examples to get some idea of what it can do, it is up to you to figure out how to use it for the remainder of the assignment.

First let's look at the number of documents, since Pyndri indexes the documents using incremental identifiers we can simply take the lowest index and the maximum document and consider the difference:

In [3]:
print("There are %d documents in this collection." % (index.maximum_document() - index.document_base()))

There are 164597 documents in this collection.


Let's take the first document out of the collection and take a look at it:

In [4]:
example_document = index.document(index.document_base())
print(example_document)
if 1 in  example_document[0]:
    print (True)

('AP890425-0001', (1360, 192, 363, 0, 880, 0, 200, 0, 894, 412, 92160, 3, 192, 0, 363, 34, 1441, 0, 174134, 0, 200, 0, 894, 412, 2652, 0, 810, 107, 49, 4903, 420, 0, 1, 48, 35, 489, 0, 35, 687, 192, 243, 0, 249311, 1877, 0, 1651, 1174, 0, 2701, 117, 412, 0, 810, 391, 245233, 1225, 5838, 16, 0, 233156, 3496, 0, 393, 17, 0, 2435, 4819, 930, 0, 0, 200, 0, 894, 0, 22, 398, 145, 0, 3, 271, 115, 0, 1176, 2777, 292, 0, 725, 192, 0, 0, 50046, 0, 1901, 1130, 0, 192, 0, 408, 0, 243779, 0, 0, 553, 192, 0, 363, 0, 3747, 0, 0, 0, 0, 1176, 0, 1239, 0, 0, 1115, 17, 0, 0, 585, 192, 1963, 0, 0, 412, 54356, 0, 773, 0, 0, 0, 192, 0, 0, 1130, 0, 363, 0, 545, 192, 0, 1174, 1901, 1130, 0, 4, 398, 145, 39, 0, 577, 0, 355, 0, 491, 0, 6025, 0, 0, 193156, 88, 34, 437, 0, 0, 1852, 0, 828, 0, 1588, 0, 0, 0, 2615, 0, 0, 107, 49, 420, 0, 0, 190, 7, 714, 2701, 0, 237, 192, 157, 0, 412, 34, 437, 0, 0, 200, 6025, 26, 0, 0, 0, 0, 363, 0, 22, 398, 145, 0, 200, 638, 126222, 6018, 0, 880, 0, 0, 161, 0, 0, 319, 894, 2701, 

TypeError: 'in <string>' requires string as left operand, not int

Here we see a document consists of two things, a string representing the external document identifier and an integer list representing the identifiers of words that make up the document. Pyndri uses integer representations for words or terms, thus a token_id is an integer that represents a word whereas the token is the actual text of the word/term. Every id has a unique token and vice versa with the exception of stop words: words so common that there are uninformative, all of these receive the zero id.

To see what some ids and their matching tokens we take a look at the dictionary of the index:

In [5]:
token2id, id2token, _ = index.get_dictionary()
print(list(id2token.items())[:15])

[(1, 'new'), (2, 'percent'), (3, 'two'), (4, '1'), (5, 'people'), (6, 'million'), (7, '000'), (8, 'government'), (9, 'president'), (10, 'years'), (11, 'state'), (12, '2'), (13, 'states'), (14, 'three'), (15, 'time')]


In [6]:
example_document[1]

(1360,
 192,
 363,
 0,
 880,
 0,
 200,
 0,
 894,
 412,
 92160,
 3,
 192,
 0,
 363,
 34,
 1441,
 0,
 174134,
 0,
 200,
 0,
 894,
 412,
 2652,
 0,
 810,
 107,
 49,
 4903,
 420,
 0,
 1,
 48,
 35,
 489,
 0,
 35,
 687,
 192,
 243,
 0,
 249311,
 1877,
 0,
 1651,
 1174,
 0,
 2701,
 117,
 412,
 0,
 810,
 391,
 245233,
 1225,
 5838,
 16,
 0,
 233156,
 3496,
 0,
 393,
 17,
 0,
 2435,
 4819,
 930,
 0,
 0,
 200,
 0,
 894,
 0,
 22,
 398,
 145,
 0,
 3,
 271,
 115,
 0,
 1176,
 2777,
 292,
 0,
 725,
 192,
 0,
 0,
 50046,
 0,
 1901,
 1130,
 0,
 192,
 0,
 408,
 0,
 243779,
 0,
 0,
 553,
 192,
 0,
 363,
 0,
 3747,
 0,
 0,
 0,
 0,
 1176,
 0,
 1239,
 0,
 0,
 1115,
 17,
 0,
 0,
 585,
 192,
 1963,
 0,
 0,
 412,
 54356,
 0,
 773,
 0,
 0,
 0,
 192,
 0,
 0,
 1130,
 0,
 363,
 0,
 545,
 192,
 0,
 1174,
 1901,
 1130,
 0,
 4,
 398,
 145,
 39,
 0,
 577,
 0,
 355,
 0,
 491,
 0,
 6025,
 0,
 0,
 193156,
 88,
 34,
 437,
 0,
 0,
 1852,
 0,
 828,
 0,
 1588,
 0,
 0,
 0,
 2615,
 0,
 0,
 107,
 49,
 420,
 0,
 0,
 190,
 7,
 71

Using this dictionary we can see the tokens for the (non-stop) words in our example document:

In [7]:
print([id2token[word_id] for word_id in example_document[1] if word_id > 0])

['52', 'students', 'arrested', 'takeover', 'university', 'massachusetts', 'building', 'fifty', 'two', 'students', 'arrested', 'tuesday', 'evening', 'occupying', 'university', 'massachusetts', 'building', 'overnight', 'protest', 'defense', 'department', 'funded', 'research', 'new', 'york', 'city', 'thousands', 'city', 'college', 'students', 'got', 'unscheduled', 'holiday', 'demonstrators', 'occupied', 'campus', 'administration', 'building', 'protest', 'possible', 'tuition', 'increases', 'prompting', 'officials', 'suspend', 'classes', '60', 'police', 'riot', 'gear', 'arrived', 'university', 'massachusetts', '5', 'p', 'm', 'two', 'hours', 'later', 'bus', 'drove', 'away', '29', 'students', 'camped', 'memorial', 'hall', 'students', 'charged', 'trespassing', '23', 'students', 'arrested', 'lying', 'bus', 'prevent', 'leaving', 'police', '300', 'students', 'stood', 'building', 'chanting', 'looking', 'students', 'hall', 'arrested', '35', 'students', 'occupied', 'memorial', 'hall', '1', 'p', 'm',

The reverse can also be done, say we want to look for news about the "University of Massachusetts", the tokens of that query can be converted to ids using the reverse dictionary:

In [8]:
query_tokens = index.tokenize("University of Massachusetts")
print("Query by tokens:", query_tokens)
query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
print("Query by ids with stopwords:", query_id_tokens)
query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]
print("Query by ids without stopwords:", query_id_tokens)

Query by tokens: ['university', '', 'massachusetts']
Query by ids with stopwords: [200, 0, 894]
Query by ids without stopwords: [200, 894]


Naturally we can now match the document and query in the id space, let's see how often a word from the query occurs in our example document:

In [9]:
matching_words = sum([True for word_id in example_document[1] if word_id in query_id_tokens])
print("Document %s has %d word matches with query: \"%s\"." % (example_document[0], matching_words, ' '.join(query_tokens)))
print("Document %s and query \"%s\" have a %.01f%% overlap." % (example_document[0], ' '.join(query_tokens),matching_words/float(len(example_document[1]))*100))

Document AP890425-0001 has 13 word matches with query: "university  massachusetts".
Document AP890425-0001 and query "university  massachusetts" have a 2.5% overlap.


While this is certainly not everything Pyndri can do, it should give you an idea of how to use it. Please take a look at the [examples](https://github.com/cvangysel/pyndri) as it will help you a lot with this assignment.

**CAUTION**: Avoid printing out the whole index in this Notebook as it will generate a lot of output and is likely to corrupt the Notebook.

### Parsing the query file
You can parse the query file (`ap_88_89/topics_title`) using the following snippet:

In [10]:
import collections
import io
import logging
import sys

def parse_topics(file_or_files,
                 max_topics=sys.maxsize, delimiter=';'):
    assert max_topics >= 0 or max_topics is None

    topics = collections.OrderedDict()

    if not isinstance(file_or_files, list) and \
            not isinstance(file_or_files, tuple):
        if hasattr(file_or_files, '__iter__'):
            file_or_files = list(file_or_files)
        else:
            file_or_files = [file_or_files]

    for f in file_or_files:
        assert isinstance(f, io.IOBase)

        for line in f:
            assert(isinstance(line, str))

            line = line.strip()

            if not line:
                continue

            topic_id, terms = line.split(delimiter, 1)

            if topic_id in topics and (topics[topic_id] != terms):
                    logging.error('Duplicate topic "%s" (%s vs. %s).',
                                  topic_id,
                                  topics[topic_id],
                                  terms)

            topics[topic_id] = terms

            if max_topics > 0 and len(topics) >= max_topics:
                break

    return topics

with open('./ap_88_89/topics_title', 'r') as f_topics:
    print(parse_topics([f_topics]))

OrderedDict([('51', 'Airbus Subsidies'), ('52', 'South African Sanctions'), ('53', 'Leveraged Buyouts'), ('54', 'Satellite Launch Contracts'), ('55', 'Insider Trading'), ('56', 'Prime (Lending) Rate Moves, Predictions'), ('57', 'MCI'), ('58', 'Rail Strikes'), ('59', 'Weather Related Fatalities'), ('60', 'Merit-Pay vs. Seniority'), ('61', 'Israeli Role in Iran-Contra Affair'), ('62', "Military Coups D'etat"), ('63', 'Machine Translation'), ('64', 'Hostage-Taking'), ('65', 'Information Retrieval Systems'), ('66', 'Natural Language Processing'), ('67', 'Politically Motivated Civil Disturbances'), ('68', 'Health Hazards from Fine-Diameter Fibers'), ('69', 'Attempts to Revive the SALT II Treaty'), ('70', 'Surrogate Motherhood'), ('71', 'Border Incursions'), ('72', 'Demographic Shifts in the U.S.'), ('73', 'Demographic Shifts across National Boundaries'), ('74', 'Conflicting Policy'), ('75', 'Automation'), ('76', 'U.S. Constitution - Original Intent'), ('77', 'Poaching'), ('78', 'Greenpeace'

### Task 1: Implement and compare lexical IR methods [40 points] ### 

In this task you will implement a number of lexical methods for IR using the **Pyndri** framework. Then you will evaluate these methods on the dataset we have provided using **TREC Eval**.

Use the **Pyndri** framework to get statistics of the documents (term frequency, document frequency, collection frequency; **you are not allowed to use the query functionality of Pyndri**) and implement the following scoring methods in **Python**:

- [TF-IDF](http://nlp.stanford.edu/IR-book/html/htmledition/tf-idf-weighting-1.html). **[5 points]**
- [BM25](http://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html) with k1=1.2 and b=0.75. **[5 points]**
- Language models ([survey](https://drive.google.com/file/d/0B-zklbckv9CHc0c3b245UW90NE0/view))
    - Jelinek-Mercer (explore different values of 𝛌 in the range [0.1, 0.5, 0.9]). **[10 points]**
    - Dirichlet Prior (explore different values of 𝛍 [500, 1000, 1500]). **[5 points]**
    - Absolute discounting (explore different values of 𝛅 in the range [0.1, 0.5, 0.9]). **[5 points]**
    
Implement the above methods and report evaluation measures (on the test set) using the hyper parameter values you optimized on the validation set (also report the values of the hyper parameters). Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

For the language models, create plots showing `NDCG@10` with varying values of the parameters. You can do this by chaining small scripts using shell scripting (preferred) or execute trec_eval using Python's `subprocess`.

Compute significance of the results using a [two-tailed paired Student t-test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html) **[10 points]**. Be wary of false rejection of the null hypothesis caused by the [multiple comparisons problem](https://en.wikipedia.org/wiki/Multiple_comparisons_problem). There are multiple ways to mitigate this problem and it is up to you to choose one.

Analyse the results by identifying specific queries where different methods succeed or fail and discuss possible reasons that cause these differences. This is *very important* in order to understand how the different retrieval functions behave.

**NOTE**: Don’t forget to use log computations in your calculations to avoid underflows. 

**IMPORTANT**: You should structure your code around the helper functions we provide below.

In [11]:
import time

start_time=time.time() 

with open('./ap_88_89/topics_title', 'r') as f_topics:
    queries = parse_topics([f_topics])

index = pyndri.Index('index/')

num_documents = index.maximum_document() - index.document_base()

dictionary = pyndri.extract_dictionary(index)

tokenized_queries = {
    query_id: [dictionary.translate_token(token)
               for token in index.tokenize(query_string)
               if dictionary.has_token(token)]
    for query_id, query_string in queries.items()}

query_term_ids = set(
    query_term_id
    for query_term_ids in tokenized_queries.values()
    for query_term_id in query_term_ids)

print('Gathering statistics about', len(query_term_ids), 'terms.')

# inverted index creation.

document_lengths = {}
unique_terms_per_document = {}

inverted_index = collections.defaultdict(dict)
collection_frequencies = collections.defaultdict(int)

total_terms = 0

for int_doc_id in range(index.document_base(), index.maximum_document()):
    ext_doc_id, doc_token_ids = index.document(int_doc_id)

    document_bow = collections.Counter(
        token_id for token_id in doc_token_ids
        if token_id > 0)
    document_length = sum(document_bow.values())

    document_lengths[int_doc_id] = document_length
    total_terms += document_length

    unique_terms_per_document[int_doc_id] = len(document_bow)

    for query_term_id in query_term_ids:
        assert query_term_id is not None

        document_term_frequency = document_bow.get(query_term_id, 0)

        if document_term_frequency == 0:
            continue

        collection_frequencies[query_term_id] += document_term_frequency
        inverted_index[query_term_id][int_doc_id] = document_term_frequency

avg_doc_length = total_terms / num_documents

print('Inverted index creation took', time.time() - start_time, 'seconds.')

Gathering statistics about 456 terms.
Inverted index creation took 97.50359392166138 seconds.


In [40]:
inverted_index[1]

{1: 2,
 131074: 1,
 3: 1,
 5: 1,
 6: 2,
 8: 8,
 9: 2,
 10: 3,
 11: 1,
 131084: 4,
 14: 1,
 131087: 1,
 16: 2,
 17: 1,
 131090: 3,
 131075: 1,
 131093: 2,
 22: 1,
 131095: 1,
 131096: 1,
 25: 1,
 154852: 2,
 29: 2,
 30: 1,
 31: 1,
 131104: 3,
 131105: 1,
 34: 2,
 35: 3,
 36: 1,
 40: 1,
 131113: 5,
 42: 3,
 43: 2,
 131116: 1,
 131117: 3,
 46: 1,
 47: 1,
 131120: 6,
 131121: 1,
 131122: 1,
 51: 3,
 131124: 6,
 53: 1,
 55: 1,
 131128: 3,
 57: 2,
 58: 3,
 131131: 3,
 131132: 1,
 131082: 1,
 62: 1,
 131135: 6,
 131083: 4,
 131140: 1,
 69: 2,
 71: 2,
 73: 1,
 131146: 3,
 152930: 3,
 78: 1,
 79: 1,
 82: 1,
 152931: 2,
 131086: 2,
 86: 4,
 88: 2,
 91: 3,
 131164: 1,
 93: 1,
 131166: 4,
 95: 1,
 96: 3,
 131088: 2,
 131171: 1,
 100: 1,
 131089: 2,
 104: 2,
 105: 1,
 106: 1,
 131179: 3,
 131180: 1,
 131181: 1,
 131182: 5,
 131183: 2,
 131184: 1,
 131185: 1,
 114: 2,
 131091: 2,
 116: 1,
 118: 1,
 119: 1,
 131192: 2,
 122: 3,
 131195: 1,
 126: 1,
 127: 1,
 128: 1,
 129: 1,
 131202: 3,
 131203: 1,
 

In [ ]:
unique_terms_per_document

In [49]:
collection_frequencies[3066]#how frequent is a word of a query inside a document

2608

In [ ]:
query_id_tokens()

In [31]:
len(inverted_index) #inverted index is only for queries

462

In [51]:
list(inverted_index[3066].keys())

[24576,
 65538,
 57351,
 143371,
 24589,
 118798,
 148394,
 81941,
 77846,
 81944,
 116888,
 151580,
 29,
 86046,
 98336,
 45744,
 71003,
 58513,
 106537,
 77886,
 12351,
 143428,
 80473,
 90186,
 86091,
 53325,
 45142,
 77912,
 69722,
 82012,
 61533,
 41056,
 69729,
 110693,
 8294,
 133137,
 110696,
 110697,
 155754,
 94315,
 143468,
 143470,
 90227,
 94324,
 118901,
 94333,
 118910,
 49281,
 139394,
 28803,
 77960,
 118921,
 53388,
 45197,
 73873,
 147602,
 163992,
 24729,
 65692,
 65693,
 159903,
 118946,
 98469,
 24745,
 24747,
 147630,
 106525,
 96298,
 143541,
 12473,
 69820,
 57535,
 123073,
 114884,
 20677,
 114887,
 41161,
 114891,
 57549,
 106702,
 41168,
 61662,
 146128,
 229,
 230,
 231,
 86248,
 4329,
 131306,
 61675,
 90348,
 28909,
 139511,
 53496,
 127225,
 139514,
 78077,
 49408,
 41220,
 119045,
 61702,
 135309,
 25987,
 139541,
 106780,
 4381,
 119070,
 155935,
 49442,
 24868,
 28969,
 155946,
 74029,
 150238,
 90423,
 12600,
 143673,
 37178,
 41279,
 24896,
 151874,

In [29]:
document_lengths['AP890425-0001']

KeyError: 'AP890425-0001'

In [32]:
len(document_lengths)

164597

In [32]:
document_lengths

{1: 296,
 2: 228,
 3: 64,
 4: 63,
 5: 246,
 6: 393,
 7: 390,
 8: 460,
 9: 278,
 10: 410,
 11: 392,
 12: 332,
 13: 520,
 14: 240,
 15: 93,
 16: 229,
 17: 250,
 18: 285,
 19: 64,
 20: 274,
 21: 91,
 22: 339,
 23: 135,
 24: 375,
 25: 270,
 26: 301,
 27: 263,
 28: 244,
 29: 222,
 30: 354,
 31: 301,
 32: 267,
 33: 359,
 34: 273,
 35: 288,
 36: 329,
 37: 282,
 38: 187,
 39: 16,
 40: 362,
 41: 242,
 42: 297,
 43: 266,
 44: 193,
 45: 224,
 46: 120,
 47: 299,
 48: 177,
 49: 88,
 50: 440,
 51: 244,
 52: 105,
 53: 104,
 54: 196,
 55: 239,
 56: 333,
 57: 315,
 58: 215,
 59: 281,
 60: 113,
 61: 142,
 62: 79,
 63: 443,
 64: 288,
 65: 291,
 66: 242,
 67: 176,
 68: 155,
 69: 246,
 70: 187,
 71: 219,
 72: 135,
 73: 355,
 74: 179,
 75: 400,
 76: 443,
 77: 198,
 78: 184,
 79: 129,
 80: 398,
 81: 173,
 82: 212,
 83: 251,
 84: 57,
 85: 362,
 86: 496,
 87: 130,
 88: 190,
 89: 201,
 90: 371,
 91: 139,
 92: 163,
 93: 100,
 94: 146,
 95: 348,
 96: 334,
 97: 79,
 98: 78,
 99: 355,
 100: 89,
 101: 451,
 102: 186

In [40]:
import numpy as np

round(np.array(list(document_lengths.values())).mean())


256.0

In [41]:
query_term_ids #are the ids of the words inside the queries

{1,
 8,
 11,
 13,
 19,
 20,
 21,
 33,
 35,
 38,
 39,
 54,
 55,
 57,
 68,
 69,
 72,
 75,
 77,
 78,
 80,
 82,
 85,
 92,
 94,
 97,
 107,
 110,
 114,
 118,
 119,
 128,
 138,
 140,
 142,
 152,
 156,
 163,
 166,
 168,
 170,
 180,
 186,
 191,
 192,
 196,
 203,
 205,
 213,
 214,
 219,
 229,
 235,
 236,
 238,
 241,
 248,
 252,
 255,
 258,
 269,
 274,
 276,
 277,
 281,
 303,
 307,
 310,
 313,
 325,
 328,
 335,
 341,
 351,
 358,
 360,
 373,
 377,
 380,
 382,
 388,
 389,
 390,
 391,
 392,
 395,
 397,
 416,
 420,
 431,
 436,
 447,
 448,
 451,
 463,
 472,
 483,
 484,
 485,
 496,
 515,
 518,
 519,
 520,
 523,
 539,
 580,
 588,
 590,
 601,
 609,
 625,
 629,
 631,
 642,
 654,
 659,
 684,
 690,
 692,
 694,
 719,
 735,
 738,
 746,
 774,
 776,
 781,
 789,
 804,
 811,
 812,
 819,
 824,
 829,
 832,
 846,
 851,
 862,
 881,
 917,
 922,
 942,
 946,
 955,
 982,
 985,
 1000,
 1033,
 1042,
 1056,
 1070,
 1098,
 1099,
 1116,
 1122,
 1138,
 1145,
 1160,
 1161,
 1178,
 1189,
 1240,
 1266,
 1268,
 1275,
 1302,
 1312,

In [41]:
queries.items()

odict_items([('51', 'Airbus Subsidies'), ('52', 'South African Sanctions'), ('53', 'Leveraged Buyouts'), ('54', 'Satellite Launch Contracts'), ('55', 'Insider Trading'), ('56', 'Prime (Lending) Rate Moves, Predictions'), ('57', 'MCI'), ('58', 'Rail Strikes'), ('59', 'Weather Related Fatalities'), ('60', 'Merit-Pay vs. Seniority'), ('61', 'Israeli Role in Iran-Contra Affair'), ('62', "Military Coups D'etat"), ('63', 'Machine Translation'), ('64', 'Hostage-Taking'), ('65', 'Information Retrieval Systems'), ('66', 'Natural Language Processing'), ('67', 'Politically Motivated Civil Disturbances'), ('68', 'Health Hazards from Fine-Diameter Fibers'), ('69', 'Attempts to Revive the SALT II Treaty'), ('70', 'Surrogate Motherhood'), ('71', 'Border Incursions'), ('72', 'Demographic Shifts in the U.S.'), ('73', 'Demographic Shifts across National Boundaries'), ('74', 'Conflicting Policy'), ('75', 'Automation'), ('76', 'U.S. Constitution - Original Intent'), ('77', 'Poaching'), ('78', 'Greenpeace'

In [49]:
def run_retrieval(model_name, score_fn, **args):
    """
    Runs a retrieval method for all the queries and writes the TREC-friendly results in a file.
    
    :param model_name: the name of the model (a string)
    :param score_fn: the scoring function (a function - see below for an example) 
    """
    run_out_path = '{}.run'.format(model_name)

    if os.path.exists(run_out_path):
        return

    retrieval_start_time = time.time()

    print('Retrieving using', model_name)

    data = {}
    
    for qid in tokenized_queries.keys(): #for each query
#     for qid in ['135']: #test

        score = {}
        for query_term_id in tokenized_queries[qid]: #for each word_id in query

            for (int_document_id,document_term_freq) in zip(inverted_index[query_term_id].keys(),inverted_index[query_term_id].values()):
                    #for each document where the word appears, calculate scoring function increment its score
                new_score = score_fn(int_document_id, query_term_id, document_term_freq, **args)
                
                if int_document_id not in score.keys():
                    #print('initiallizing and incrementing score')
                    score[int_document_id] = new_score
                else:
                    #print('incrementing score')
                    score[int_document_id] += new_score

        #turn dict to list of tuples and continue        
        # The dictionary data should have the form: query_id --> (document_score, external_doc_id)
        data[qid] = [(score, index.ext_document_id(doc_id)) for (score, doc_id) in zip(score.values(), score.keys())]
        
#     write out
    with open(run_out_path, 'w') as f_out:
        write_run(
            model_name=model_name,
            data=data,
            out_f=f_out,
            max_objects_per_query=1000)
    

In [13]:
def test_fn(int_document_id, query_term_id, document_term_freq):
    #test function returning the occurencies themselves as a score
    #return inverted_index[query_term_id][int_document_id]
    return 1.

run_retrieval('test',test_fn)

Retrieving using test


KeyboardInterrupt: 

In [16]:
import numpy as np

### TF-IDF

In [17]:
def tfidf(int_document_id, query_term_id, document_term_freq):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query term id (assuming you have split the query to tokens)
    :param document_term_freq: the document term frequency of the query term 
    """
    
    ## Amount of time a term appears in a document  
    df = document_term_freq
    ## Total terms in a document
    dl = document_lengths[int_document_id]
    ## Amount of documents containing a term
    n = len(inverted_index[query_term_id])
    
    tf = np.log2(1 + (df/dl))
    idf = np.log2((n/df))
    
    score = tf * idf

    return score




In [18]:
run_retrieval('tf-idf',tfidf)

Retrieving using tf-idf


### BM25

In [19]:
def bm25(int_document_id, query_term_id, document_term_freq ,k1=1.2 , b=0.75,length_avg = round(np.average(np.array(list(document_lengths.values())))) ):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query term id (assuming you have split the query to tokens)
    :param document_term_freq: the document term frequency of the query term
    
    Hyperparameters: k1=1.2 and b=0.75
    
    """
    
    ## Amount of time a term appears in a document  
    df = document_term_freq
     
    
    ## tf, idf
    tf = np.log2(1 + (df/document_lengths[int_document_id]))
    idf = np.log2((len(inverted_index[query_term_id])/df))
    
    ## Total terms in a document : document_lengths[int_document_id]    
    ## Amount of documents containing a term : len(inverted_index[query_term_id])
    ## document length : document_lengths[int_document_id]
    
    return ((k1+1)*tf*idf)/(k1*((1-b)+b*(document_lengths[int_document_id] /length_avg))+tf)


In [20]:
start_time=time.time() 

run_retrieval('bm25',bm25)

print('BM25 scoring took', time.time() - start_time, 'seconds.')

Retrieving using bm25
BM25 scoring took 76.56085014343262 seconds.


In [21]:
##trec_eval command for evalutaion of bm25

!trec_eval -m all_trec -q ./ap_88_89/qrel_validation bm25.run | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"

P_5                   	103	0.0000
recall_1000           	103	0.5636
ndcg_cut_10           	103	0.0000
map_cut_1000          	103	0.0276
P_5                   	111	0.0000
recall_1000           	111	0.5982
ndcg_cut_10           	111	0.0000
map_cut_1000          	111	0.0477
P_5                   	114	0.0000
recall_1000           	114	0.3252
ndcg_cut_10           	114	0.1428
map_cut_1000          	114	0.0326
P_5                   	120	0.2000
recall_1000           	120	0.0750
ndcg_cut_10           	120	0.0948
map_cut_1000          	120	0.0111
P_5                   	123	0.0000
recall_1000           	123	0.0000
ndcg_cut_10           	123	0.0000
map_cut_1000          	123	0.0000
P_5                   	135	0.0000
recall_1000           	135	0.2800
ndcg_cut_10           	135	0.0000
map_cut_1000          	135	0.0066
P_5                   	143	0.0000
recall_1000           	143	0.0812
ndcg_cut_10           	143	0.0000
map_cut_1000          	143	0.0022
P_5                   	144	0.0000
recall_1000   

### Jelinek-Mercer

In [22]:
#precompute the 2nd term of jelinek mercer smoothening to speed-up computations
P_mle_col = dict()
for query_term_id in query_term_ids:
    P_mle_col[query_term_id] = index.get_term_frequencies()[query_term_id] / index.total_terms()

In [ ]:
def jelinek_mercer(int_document_id, query_term_id, document_term_freq, lambd = 0.75):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query term id (assuming you have split the query to tokens)
    :param document_term_freq: the document term frequency of the query term
    :param lambd: lambda value of linear interpolation (default = 0.75)
    """
    P_mle_doc = document_term_freq / document_lengths[int_document_id]
    
    return ( (lambd * P_mle_doc) + ((1-lambd)*P_mle_col[query_term_id]) )

In [ ]:
#final run
start_time = time.time()
test_dict = run_retrieval('jelinek_mercer',jelinek_mercer)
print ('elapsed time:',time.time()-start_time)

In [ ]:
#trec_eval command for evalutaion of jelinek_mercer

!trec_eval -m all_trec -q ./ap_88_89/qrel_validation jelinek_mercer.run | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"

### Dirichlet Prior

In [23]:
def dirichlet_prior(int_document_id, query_term_id, document_term_freq, mu=500):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query term id (assuming you have split the query to tokens)
    :param document_term_freq: the document term frequency of the query term 
    """
    ## Amount of time a term appears in a document  
    df = document_term_freq
     
    
    ## tf, idf
    tf = np.log2(1 + (df/document_lengths[int_document_id]))
    
    
    ##p_w_C = P_mle_col[query_term_id])
    ## document length : document_lengths[int_document_id]
    
    
    
    
    return (tf+mu*P_mle_col[query_term_id])/(document_lengths[int_document_id]+mu)


In [70]:
#final run
start_time = time.time()
test_dict = run_retrieval('dirichlet_prior',dirichlet_prior, mu = 500)
print ('elapsed time:',time.time()-start_time)

Retrieving using dirichlet_prior
elapsed time: 70.12480020523071


In [71]:
#trec_eval command for evalutaion of dirichlet_prior

!trec_eval -m all_trec -q ./ap_88_89/qrel_validation dirichlet_prior.run | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"

P_5                   	103	0.4000
recall_1000           	103	0.3273
ndcg_cut_10           	103	0.2583
map_cut_1000          	103	0.0447
P_5                   	111	0.2000
recall_1000           	111	0.5089
ndcg_cut_10           	111	0.2201
map_cut_1000          	111	0.0572
P_5                   	114	0.0000
recall_1000           	114	0.0813
ndcg_cut_10           	114	0.0000
map_cut_1000          	114	0.0015
P_5                   	120	0.2000
recall_1000           	120	0.0750
ndcg_cut_10           	120	0.2201
map_cut_1000          	120	0.0270
P_5                   	123	0.0000
recall_1000           	123	0.0571
ndcg_cut_10           	123	0.0000
map_cut_1000          	123	0.0003
P_5                   	135	0.2000
recall_1000           	135	0.1067
ndcg_cut_10           	135	0.1100
map_cut_1000          	135	0.0068
P_5                   	143	0.0000
recall_1000           	143	0.0295
ndcg_cut_10           	143	0.0000
map_cut_1000          	143	0.0006
P_5                   	144	0.0000
recall_1000   

In [72]:
!trec_eval -m all_trec -q ./ap_88_89/qrel_validation dirichlet_prior.run | grep -E "^ndcg_cut_10\s"

ndcg_cut_10           	103	0.2583
ndcg_cut_10           	111	0.2201
ndcg_cut_10           	114	0.0000
ndcg_cut_10           	120	0.2201
ndcg_cut_10           	123	0.0000
ndcg_cut_10           	135	0.1100
ndcg_cut_10           	143	0.0000
ndcg_cut_10           	144	0.0000
ndcg_cut_10           	151	0.3571
ndcg_cut_10           	155	0.0000
ndcg_cut_10           	158	0.0000
ndcg_cut_10           	165	0.0000
ndcg_cut_10           	167	0.3527
ndcg_cut_10           	170	0.3487
ndcg_cut_10           	173	1.0000
ndcg_cut_10           	180	0.0000
ndcg_cut_10           	182	0.0784
ndcg_cut_10           	192	0.2083
ndcg_cut_10           	53	0.0000
ndcg_cut_10           	57	0.4634
ndcg_cut_10           	69	0.0000
ndcg_cut_10           	74	0.0000
ndcg_cut_10           	78	1.0000
ndcg_cut_10           	86	0.0000
ndcg_cut_10           	89	0.0000
ndcg_cut_10           	90	0.4000
ndcg_cut_10           	92	0.0000
ndcg_cut_10           	93	0.0000
ndcg_cut_10           	94	0.0734
ndcg_cut_10           	95

In [76]:
score = !trec_eval -m all_trec -q ./ap_88_89/qrel_validation dirichlet_prior.run | grep -E "^ndcg_cut_10\s"

print  (score[-1])
print(score[-1].split()[-1])

ndcg_cut_10           	all	0.1697
0.1697


### Absolute discounting

In [26]:
def absolute_discounting(int_document_id, query_term_id, document_term_freq, delta=0.1):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query term id (assuming you have split the query to tokens)
    :param document_term_freq: the document term frequency of the query term 
    """
    ## Amount of time a term appears in a document  
    df = document_term_freq
     
    
    ## tf, idf
    tf = np.log2(1 + (df/document_lengths[int_document_id]))
    
    ## p_w_C = P_mle_col[query_term_id])
    ## document length : document_lengths[int_document_id]
    ## document length u: collection_frequencies[query_term_id]
    
    doc_length = document_lengths[int_document_id]
    
    return ((np.max(tf-delta,0)/doc_length)+
            ((delta*collection_frequencies[query_term_id]*P_mle_col[query_term_id])/doc_length))

In [27]:
#final run
start_time = time.time()
test_dict = run_retrieval('absolute_discounting',absolute_discounting)
print ('elapsed time:',time.time()-start_time)

Retrieving using absolute_discounting
elapsed time: 101.31073212623596


In [28]:
#trec_eval command for evalutaion of dirichlet_prior

!trec_eval -m all_trec -q ./ap_88_89/qrel_validation absolute_discounting.run | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"

P_5                   	103	0.0000
recall_1000           	103	0.0182
ndcg_cut_10           	103	0.0000
map_cut_1000          	103	0.0000
P_5                   	111	0.0000
recall_1000           	111	0.1250
ndcg_cut_10           	111	0.0000
map_cut_1000          	111	0.0017
P_5                   	114	0.0000
recall_1000           	114	0.0000
ndcg_cut_10           	114	0.0000
map_cut_1000          	114	0.0000
P_5                   	120	0.0000
recall_1000           	120	0.0750
ndcg_cut_10           	120	0.0000
map_cut_1000          	120	0.0013
P_5                   	123	0.0000
recall_1000           	123	0.0000
ndcg_cut_10           	123	0.0000
map_cut_1000          	123	0.0000
P_5                   	135	0.0000
recall_1000           	135	0.0000
ndcg_cut_10           	135	0.0000
map_cut_1000          	135	0.0000
P_5                   	143	0.0000
recall_1000           	143	0.0037
ndcg_cut_10           	143	0.0000
map_cut_1000          	143	0.0000
P_5                   	144	0.0000
recall_1000   

In [42]:
name = 'absolute_discounting.run'
def test(name):



    test = !trec_eval -m all_trec -q ./ap_88_89/qrel_validation name | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"


In [43]:
test

<function __main__.test>

In [37]:
print(test[-4:])
print (test[-2].split()[-1])

['P_5                   \tall\t0.0800', 'recall_1000           \tall\t0.1714', 'ndcg_cut_10           \tall\t0.0860', 'map_cut_1000          \tall\t0.0491']
0.0860


In [46]:
lamda = [0.1, 0.5, 0.9]
mu = [500, 1000, 1500]
delta = [0.1, 0.5, 0.9]

In [79]:
import os

def plot_ndcg10_jelinek_mercer(hyperparameters):
    '''Returns a dictionary with the hyperparametes as keys and the ndcg@10 score as values
       of jelinek_mercer.
    '''
    scores_dict={}
    
    for i in hyperparameters:
    
        test1_dict = run_retrieval('jelinek_mercer',jelinek_mercer,lambd = i)
        score = !trec_eval -m all_trec -q ./ap_88_89/qrel_validation jelinek_mercer.run | grep -E "^ndcg_cut_10\s"
        scores_dict[str(i)] = score[-1].split()[-1]

        
    return scores_dict
        
def plot_ndcg10_dirichlet_prior(hyperparameters):
    '''Returns a dictionary with the hyperparametes as keys and the ndcg@10 score as values
       of dirichlet_prior.
    '''
    
    scores_dict={}
    
    for i in hyperparameters:
        os.remove('dirichlet_prior.run')
        print(i)
        run_retrieval('dirichlet_prior',dirichlet_prior, mu = i)
        score = !trec_eval -m all_trec -q ./ap_88_89/qrel_validation dirichlet_prior.run | grep -E "^ndcg_cut_10\s"
        print(score[-1].split()[-1])
        scores_dict[str(i)] = score[-1].split()[-1]

        
    return scores_dict
        
def plot_ndcg10_absolute_discounting(hyperparameters):
    '''Returns a dictionary with the hyperparametes as keys and the ndcg@10 score as values
       of absolute_discounting.
    '''
    
    scores_dict={}
    
    for i in hyperparameters:
    
        run_retrieval('absolute_discounting',absolute_discounting,delta = i)
        score = !trec_eval -m all_trec -q ./ap_88_89/qrel_validation absolute_discounting.run | grep -E "^ndcg_cut_10\s"
        scores_dict[str(i)] = score[-1].split()[-1]
        
    return scores_dict
        

In [80]:
test_dict = plot_ndcg10_dirichlet_prior(mu)

500
Retrieving using dirichlet_prior
0.1697
1000
Retrieving using dirichlet_prior
0.1998
1500
Retrieving using dirichlet_prior
0.2106


In [57]:
test_dict

{'1000': '0.0000', '1500': '0.0000', '500': '0.0000'}

### Task 2: Latent Semantic Models (LSMs) [20 points] ###

In this task you will experiment with applying distributional semantics methods ([LSI](http://lsa3.colorado.edu/papers/JASIS.lsi.90.pdf) **[5 points]** and [LDA](https://www.cs.princeton.edu/~blei/papers/BleiNgJordan2003.pdf) **[5 points]**) for retrieval.

You do not need to implement LSI or LDA on your own. Instead, you can use [gensim](http://radimrehurek.com/gensim/index.html). An example on how to integrate Pyndri with Gensim for word2vec can be found [here](https://github.com/cvangysel/pyndri/blob/master/examples/word2vec.py). For the remaining latent vector space models, you will need to implement connector classes (such as `IndriSentences`) by yourself.

In order to use a latent semantic model for retrieval, you need to:
   * build a representation of the query **q**,
   * build a representation of the document **d**,
   * calculate the similarity between **q** and **d** (e.g., cosine similarity, KL-divergence).
     
The exact implementation here depends on the latent semantic model you are using. 
   
Each of these LSMs come with various hyperparameters to tune. Make a choice on the parameters, and explicitly mention the reasons that led you to these decisions. You can use the validation set to optimize hyper parameters you see fit; motivate your decisions. In addition, mention clearly how the query/document representations were constructed for each LSM and explain your choices.

In this experiment, you will first obtain an initial top-1000 ranking for each query using TF-IDF in **Task 1**, and then re-rank the documents using the LSMs. Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

Perform significance testing **[5 points]** (similar as in Task 1) in the class of semantic matching methods.

Perform analysis **[5 points]**

### Task 3:  Word embeddings for ranking [10 points] ###

First create word embeddings on the corpus we provided using [word2vec](http://arxiv.org/abs/1411.2738) -- [gensim implementation](https://radimrehurek.com/gensim/models/word2vec.html). You should extract the indexed documents using pyndri and provide them to gensim for training a model (see example [here](https://github.com/nickvosk/pyndri/blob/master/examples/word2vec.py)).

Try one of the following (increasingly complex) methods for building query and document representations:
   * Average or sum the word vectors.
   * Cluster words in the document using [k-means](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) and use the centroid of the most important cluster. Experiment with different values of K for k-means.
   * Using the [bag-of-word-embeddings representation](https://ciir-publications.cs.umass.edu/pub/web/getpdf.php?id=1248).
   
Note that since we provide the implementation for training word2vec, you will be graded based on your creativity on combining word embeddings for building query and document representations.

Note: If you want to experiment with pre-trained word embeddings on a different corpus, you can use the word embeddings we provide alongside the assignment (./data/reduced_vectors_google.txt). These are the [google word2vec word embeddings](https://code.google.com/archive/p/word2vec/), reduced to only the words that appear in the document collection we use in this assignment.

### Task 4: Learning to rank (LTR) [10 points] ###

In this task you will get an introduction into learning to rank for information retrieval, in particular pointwise learning to rank.

You will experiment with a pointwise learning to rank method, logistic regression, implemented in [scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).
Train your LTR model using 10-fold cross validation on the test set.

You can explore different ways for devising features for the model. Obviously, you can use the retrieval methods you implemented in Task 1 and Task 2 as features. Think about other features you can use (e.g. query/document length). 
One idea is to also explore external sources such as Wikipedia entities (?). Creativity on devising new features and providing motivation for them will be taken into account when grading.

For every query, first create a document candidate set using the top-1000 documents using TF-IDF, and subsequently compute features given a query and a document. Note that the feature values of different retrieval methods are likely to be distributed differently.

### Task 4: Write a report [20 points; instant FAIL if not provided] ###

The report should be a PDF file created using the [sigconf ACM template](https://www.acm.org/publications/proceedings-template) and will determine a significant part of your grade.

   * It should explain what you have implemented, motivate your experiments and detail what you expect to learn from them. **[10 points]**
   * Lastly, provide a convincing analysis of your results and conclude the report accordingly. **[10 points]**
      * Do all methods perform similarly on all queries? Why?
      * Is there a single retrieval model that outperforms all other retrieval models (i.e., silver bullet)?
      * ...

**Hand in the report and your self-contained implementation source files.** Only send us the files that matter, organized in a well-documented zip/tgz file with clear instructions on how to reproduce your results. That is, we want to be able to regenerate all your results with minimal effort. You can assume that the index and ground-truth information is present in the same file structure as the one we have provided.
